In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [10]:
dfa = pd.read_sas(input_dir+'AUX_J_2017.XPT')
dfq = pd.read_sas(input_dir+'AUQ_J_2017.XPT')
dfd = pd.read_sas(input_dir+'DEMO_J_2017.XPT')

In [11]:
dfa = dfa[['SEQN','AUXU1K1R', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU1K1L','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,100)

audiogram_col = ['AUXU1K1R', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU1K1L','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:8]
audiogram_left = audiogram_col[8:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 160) & (dfa[audiogram_right].sum(axis=1) >= 160)].reset_index(drop=True)
dfa

,SEQN,AUXU1K1R,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU1K1L,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,93709,20,5,20,30,25,25,30,35,20,15,20,30,20,20,25,25
1,93715,10,5,10,15,65,70,60,55,15,5,15,55,65,70,65,70
2,93727,5,5,-5,35,50,60,70,65,10,10,10,15,10,15,15,25
3,93732,10,15,10,40,55,75,85,85,5,10,10,30,55,70,60,70
4,93740,55,50,60,45,55,55,60,65,20,20,20,30,35,30,45,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,102890,25,25,25,70,65,75,85,80,30,35,30,75,70,90,95,100
786,102904,10,10,10,15,30,35,20,40,5,20,5,15,30,40,40,35
787,102924,25,25,25,45,55,60,55,70,15,30,15,50,50,65,60,60
788,102941,60,55,60,50,45,45,60,65,5,0,0,-5,0,0,5,10


In [12]:
dfq = dfq[['SEQN','AUQ054', 'AUQ400']]
dfq=dfq.dropna(how='any').reset_index(drop=True)
dfq.shape

(606, 3)

In [9]:
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR', 'DMQMILIZ']] #1 = male, 2 = female

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd = dfd.dropna(subset=['DMQMILIZ'], how='all').reset_index(drop=True)
dfd.shape

ValueError: bins must increase monotonically.

In [7]:
dfV = dfd.merge(dfa, on='SEQN', how='inner').merge(dfq, on='SEQN', how='inner')
print(dfV.shape)
dfV.to_excel(output_dir+"NHANES_2017-2018.xlsx")

(409, 23)


In [8]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,DMQMILIZ,DMQADFC,AUXU1K1R,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,...,AUXU1K1L,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL,AUQ054,AUQ400
0,93715.0,1.0,5,2.0,NaN,10,5,10,15,65,...,15,5,15,55,65,70,65,70,3.0,7.0
1,93732.0,1.0,5,2.0,NaN,10,15,10,40,55,...,5,10,10,30,55,70,60,70,3.0,6.0
2,93740.0,1.0,5,2.0,NaN,55,50,60,45,55,...,20,20,20,30,35,30,45,65,5.0,7.0
3,93765.0,1.0,1,2.0,NaN,75,75,85,95,95,...,95,85,95,105,90,100,100,100,4.0,2.0
4,93768.0,1.0,5,1.0,2.0,20,40,15,45,85,...,35,45,35,45,65,70,80,85,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,102807.0,1.0,5,1.0,2.0,45,20,50,60,70,...,50,25,50,95,105,100,100,100,3.0,4.0
405,102815.0,1.0,5,1.0,1.0,40,50,40,60,70,...,70,60,70,100,100,100,100,100,4.0,7.0
406,102890.0,1.0,5,1.0,1.0,25,25,25,70,65,...,30,35,30,75,70,90,95,100,4.0,6.0
407,102924.0,2.0,5,2.0,NaN,25,25,25,45,55,...,15,30,15,50,50,65,60,60,4.0,3.0
